## Imports

In [ ]:
# Code to make the surprise library work
!pip uninstall -y numpy scikit-surprise tensorflow numba
!pip install numpy==1.26.4
!pip install numba==0.60.0
!pip install scikit-surprise
!pip install tensorflow==2.18.0

In [ ]:
!pip install jupyter-dash

In [ ]:
!pip install gensim

In [ ]:
import base64
import pickle
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from gensim import corpora, models
from dash import Dash, html, dcc, Input, Output, callback_context, State
from dash.exceptions import PreventUpdate
from dash.dependencies import ALL
from wordcloud import WordCloud
from io import BytesIO
from surprise import KNNWithMeans, Dataset, Reader
from surprise import SVD

## Dashboard

In [ ]:
# ------------------------------
# Dataset definition
# ------------------------------
filename = "datasets/boardgames_4122_clean_glove_dual_tone_bert_popularity.csv"
games = pd.read_csv(filename, sep=";")
games["glove_vector"] = games["glove_vector"].apply(lambda x: np.fromstring(str(x).strip("[]"), sep=" "))
reviews = pd.read_csv("datasets/filtered_reviews.csv", sep=";")

In [ ]:
# ------------------------------
# TF-IDF
# ------------------------------
def prepare_tfidf_matrix(games):
  # Tokenize
  tokenized_descr = [clean_description.split() for clean_description in games["clean_description"]]

  # Dictionary and corpus
  dictionary = corpora.Dictionary(tokenized_descr)
  bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_descr]

  # TF-IDF corpus
  tfidf = models.TfidfModel(bow_corpus)
  tfidf_corpus = tfidf[bow_corpus]
  num_docs = len(tfidf_corpus)
  num_terms = len(dictionary)

  # Create empty matrix
  X_tfidf_gensim = np.zeros((num_docs, num_terms))

  # Fill it with TF-IDF scores
  for doc_idx, doc in enumerate(tfidf_corpus):
      for term_id, tfidf_score in doc:
          X_tfidf_gensim[doc_idx, term_id] = tfidf_score

  return dictionary, bow_corpus, tfidf_corpus, X_tfidf_gensim

def plot_tfidf_summary(dictionary, X_tfidf, top_n=25):
    # Average and total scores
    avg_scores = X_tfidf.mean(axis=0)
    tfidf_term_scores = X_tfidf.sum(axis=0)

    # Bar plot: top average scores
    top_indices = avg_scores.argsort()[::-1][:top_n]
    top_terms = [dictionary[i] for i in top_indices]
    top_scores = avg_scores[top_indices]

    bar_fig = go.Figure(go.Bar(
        x=top_terms,
        y=top_scores,
        marker_color='indigo'
    ))
    bar_fig.update_layout(
        xaxis_title="Term",
        yaxis_title="Average TF-IDF",
        xaxis_tickangle=-45
    )

    # Word cloud
    word_freq = {dictionary[i]: tfidf_term_scores[i] for i in range(len(dictionary))}

    wordcloud = WordCloud(
        width=800, height=400,
        background_color='white',
        colormap='viridis',
        max_words=200
    ).generate_from_frequencies(word_freq)

    # Convert to base64 image string for Dash
    img = BytesIO()
    wordcloud.to_image().save(img, format='PNG')
    img.seek(0)
    encoded_image = base64.b64encode(img.read()).decode()

    return bar_fig, encoded_image

dictionary, bow_corpus, tfidf_corpus, X_tfidf = prepare_tfidf_matrix(games)
_, encoded_image = plot_tfidf_summary(dictionary, X_tfidf)


# ------------------------------
# Popular Games
# ------------------------------
def get_diverse_popular_games(games, top_k=10):
    clusters = games["description_cluster8"].unique()
    games = games.copy()
    games = games.sort_values("popularity_score", ascending=False)

    recommended_games = pd.DataFrame()
    used_ids = set()

    while len(recommended_games) < top_k:
        added_this_round = 0

        for cluster in clusters:
            if len(recommended_games) >= top_k:
                break

            # Get next top game from this cluster that hasn't been used
            candidates = games[
                (games["description_cluster8"] == cluster) &
                (~games["id"].isin(used_ids))
            ]

            if not candidates.empty:
                top_game = candidates.iloc[0:1]
                recommended_games = pd.concat([recommended_games, top_game])
                used_ids.update(top_game["id"])
                added_this_round += 1

        # Break if no new games could be added in this pass
        if added_this_round == 0:
            break

    # Explanations
    explanations = []
    for _, row in recommended_games.iterrows():
        rating = round(row["avg_rating"], 2)
        num = int(row["num_ratings"])
        explanations.append(f"This game is highly rated ({rating}/10 from {num} users)")

    recommended_games["explanation"] = explanations

    return recommended_games[["name", "clean_description", "avg_rating", "num_ratings", "popularity_score", "explanation"]].head(top_k)

# ------------------------------
# Content-Based System (MMR)
# ------------------------------
def mmr(user_profile, candidate_vectors, candidate_ids, lambda_param=0.7, top_k=10):
    selected = []
    selected_ids = []
    candidate_indices = list(range(len(candidate_vectors)))

    similarities_to_user = cosine_similarity(candidate_vectors, [user_profile]).flatten()
    similarity_matrix = cosine_similarity(candidate_vectors)

    for _ in range(top_k):
        mmr_scores = []
        for idx in candidate_indices:
            if not selected:
                diversity_penalty = 0
            else:
                diversity_penalty = max(similarity_matrix[idx][j] for j in selected)

            mmr_score = lambda_param * similarities_to_user[idx] - (1 - lambda_param) * diversity_penalty
            mmr_scores.append((idx, mmr_score))

        selected_idx, _ = max(mmr_scores, key=lambda x: x[1])
        selected.append(selected_idx)
        selected_ids.append(candidate_ids[selected_idx])
        candidate_indices.remove(selected_idx)

    return selected_ids

def visualize_mmr(user_vector, candidate_vectors, candidate_ids, game_names, lambda_param=0.7, top_k=10):
    # Get top-k by similarity
    relevance = cosine_similarity(candidate_vectors, [user_vector]).flatten()
    top_k_indices = relevance.argsort()[::-1][:top_k]
    top_k_ids = [candidate_ids[i] for i in top_k_indices]

    # Get top-k using MMR
    mmr_ids = mmr(user_vector, candidate_vectors, candidate_ids, lambda_param, top_k)

    # Reduce dimensions for plotting
    pca = PCA(n_components=2)
    reduced = pca.fit_transform(np.vstack([user_vector, candidate_vectors]))
    user_p = reduced[0]
    candidates_p = reduced[1:]

    plt.figure(figsize=(10, 7))
    plt.scatter(candidates_p[:, 0], candidates_p[:, 1], c='lightgray', label="All Candidates", alpha=0.5)

    # Plot user profile
    plt.scatter(user_p[0], user_p[1], c='blue', label="User Profile", marker='X', s=100)

    # Plot top-k by similarity
    for idx in top_k_indices:
        plt.scatter(candidates_p[idx, 0], candidates_p[idx, 1], c='green', label="Top-k Similarity" if idx == top_k_indices[0] else "", edgecolors='black')
        plt.text(candidates_p[idx, 0], candidates_p[idx, 1], game_names[candidate_ids[idx]], fontsize=8, color='darkgreen')

    # Plot MMR-selected games
    for mmr_id in mmr_ids:
        i = candidate_ids.index(mmr_id)
        plt.scatter(candidates_p[i, 0], candidates_p[i, 1], c='orange', label="Top-k MMR" if mmr_id == mmr_ids[0] else "", edgecolors='black')
        plt.text(candidates_p[i, 0], candidates_p[i, 1], game_names[mmr_id], fontsize=8, color='darkorange')

    plt.legend()
    plt.title(f"MMR vs Top-k Similarity (λ = {lambda_param})")
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.grid(True)

    # Instead of plt.show(), convert to base64
    buf = BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight')
    plt.close()
    buf.seek(0)
    return base64.b64encode(buf.read()).decode()

def recommend_content_based_mmr(user_name, games, reviews, top_k=10, testing=False, max_playingtime=None, min_age=None, minplayers=None, maxplayers=None, min_rating=6, plot=False):
    # Filter games based on user preferences
    filtered_games = games.copy()

    if max_playingtime is not None:
        filtered_games = filtered_games[filtered_games["playingtime"] <= max_playingtime]
    if min_age is not None:
        filtered_games = filtered_games[filtered_games["minage"] >= min_age]
    if minplayers is not None:
        filtered_games = filtered_games[filtered_games["minplayers"] <= minplayers]
    if maxplayers is not None:
        filtered_games = filtered_games[filtered_games["maxplayers"] >= maxplayers]
    if min_rating is not None:
        filtered_games = filtered_games[filtered_games["rating"] >= min_rating]

    if filtered_games.empty:
        print("No games match the provided filters.")
        return

    # Get games the user liked
    liked_game_ids = reviews[(reviews["user"] == user_name) & (reviews["rating"] >= 6)]
    liked_vectors = liked_game_ids.merge(games[["id", "glove_vector"]], left_on="ID", right_on="id")

    if liked_vectors.empty:
        print("Not enough liked games with vectors to build a profile.")
        return

    # Build user profile (weighted average)
    ratings = liked_vectors["rating"].values
    weights = ratings / ratings.max()
    vectors = np.vstack(liked_vectors["glove_vector"])
    user_profile = np.average(vectors, axis=0, weights=weights)

    if not testing:
        # Remove already liked games from recommendations
        unseen_mask = ~filtered_games["id"].isin(liked_game_ids["ID"])
        filtered_games = filtered_games[unseen_mask]

    if filtered_games.empty:
        print("No new games to recommend after filtering out liked ones.")
        return

    # Prepare data for MMR
    game_vectors = np.vstack(filtered_games["glove_vector"].values)
    similarities = cosine_similarity(game_vectors, [user_profile]).flatten()

    # Apply MMR for diversity
    selected_ids = mmr(user_profile, game_vectors, filtered_games["id"].tolist(), lambda_param=0.7, top_k=top_k)
    recommended_games = filtered_games[filtered_games["id"].isin(selected_ids)].copy()

    # Add similarity score for interpretability
    recommended_games["similarity"] = similarities[[filtered_games["id"].tolist().index(i) for i in selected_ids]]

    # Generate personalized explanations with vibe and emotion
    explanations = []
    for game_id in selected_ids:
        # Find the most similar liked game
        similarities_to_liked_games = cosine_similarity([games.loc[games["id"] == game_id, "glove_vector"].values[0]], liked_vectors["glove_vector"].tolist()).flatten()
        most_similar_game_idx = np.argmax(similarities_to_liked_games)
        most_similar_game_id = liked_game_ids.iloc[most_similar_game_idx]["ID"]
        most_similar_game_name = games.loc[games["id"] == most_similar_game_id, "name"].values[0]
        most_similar_rating = liked_game_ids.iloc[most_similar_game_idx]["rating"]

        # Get predicted vibe and emotion for the recommended game
        predicted_vibe = games.loc[games["id"] == game_id, "predicted_vibe"].values[0]
        predicted_emotion = games.loc[games["id"] == game_id, "predicted_emotion"].values[0]

        # Generate the explanation
        explanation = (
            f"I recommend this game because you liked '{most_similar_game_name}' "
            f"(you rated it {most_similar_rating}/10). "
            f"This game has a '{predicted_emotion}' feeling and is '{predicted_vibe}'."
        )
        explanations.append(explanation)

    recommended_games["explanation"] = explanations

    mmr_plot_encoded = None
    if plot:
        mmr_plot_encoded = visualize_mmr(user_profile, game_vectors, filtered_games["id"].tolist(), dict(zip(games["id"], games["name"])), lambda_param=0.7, top_k=top_k)

    return recommended_games[["name", "rating", "id", "similarity", "explanation"]], mmr_plot_encoded

# ------------------------------
# Hybrid Content-Based
# ------------------------------
def predict_all_ratings(user_name, games_df, reviews_df, k=10):
    # Get user-rated games and their vectors
    user_reviews = reviews_df[reviews_df["user"] == user_name]
    if user_reviews.empty:
        return "No reviews found for this user."

    user_games = user_reviews.merge(games_df[["id", "glove_vector"]], left_on="ID", right_on="id")
    if user_games.empty:
        return "User has rated games with no vector information."

    rated_vectors = np.vstack(user_games["glove_vector"])
    rated_ratings = user_games["rating"].values
    rated_names = user_games["name"].tolist()

    # Games not yet rated by user
    unseen_games = games_df[~games_df["id"].isin(user_games["id"])].copy()
    if unseen_games.empty:
        return "No unseen games to predict ratings for."

    predicted_ratings = []

    for _, game in unseen_games.iterrows():
        target_vector = game["glove_vector"]

        # Cosine similarity with all user-rated games
        similarities = cosine_similarity(rated_vectors, [target_vector]).flatten()
        similarities = np.maximum(similarities, 0)

        # Select top-k most similar games
        if len(similarities) < k:
            k_adj = len(similarities)
        else:
            k_adj = k

        top_k_idx = np.argsort(similarities)[-k_adj:]
        top_similarities = similarities[top_k_idx]
        top_ratings = rated_ratings[top_k_idx]
        top_game_names = [rated_names[i] for i in top_k_idx]

        if top_similarities.sum() == 0:
            pred_rating = np.nan
            explanation = "No sufficiently similar games found."
        else:
            pred_rating = np.average(top_ratings, weights=top_similarities)
            # Build explanation
            if len(top_game_names) == 1:
                explanation = f"Because you liked **{top_game_names[0]}**, and this game is similar to it."
            elif len(top_game_names) == 2:
                explanation = f"Because you liked **{top_game_names[0]}** and **{top_game_names[1]}**, and this game is similar to both."
            else:
                explanation = f"Because you liked games like **{top_game_names[0]}**, **{top_game_names[1]}**, and others."

        predicted_ratings.append((game["id"], game["name"], round(pred_rating, 2) if not np.isnan(pred_rating) else None, explanation))

    # Return as a DataFrame
    pred_df = pd.DataFrame(predicted_ratings, columns=["id", "name", "predicted_rating", "explanation"])
    pred_df = pred_df.sort_values(by="predicted_rating", ascending=False)

    return pred_df.head(k)

# ------------------------------
# Collaborative functions
# ------------------------------
def recommend_with_knn(user_name, games_df, reviews_df, top_k=10):
    # Load KNNWithMeans model
    with open("models/knn_model.pkl", "rb") as f:
        model = pickle.load(f)

    # Get games the user has not rated
    rated_ids = reviews_df[reviews_df["user"] == user_name]["ID"].unique()
    unseen_ids = [gid for gid in games_df["id"].unique() if gid not in rated_ids]

    # Predict ratings
    predictions = [(gid, model.predict(user_name, gid).est) for gid in unseen_ids]
    top_preds = sorted(predictions, key=lambda x: x[1], reverse=True)[:top_k]

    # Retrieve game info
    top_game_ids = [gid for gid, _ in top_preds]
    recommended = games_df[games_df["id"].isin(top_game_ids)][["name", "rating", "id"]].copy()
    recommended["predicted_rating"] = [round(r, 2) for _, r in top_preds]

    return recommended.sort_values("predicted_rating", ascending=False)

def recommend_with_svd(user_name, games_df, reviews_df, top_k=10):
    # Load model
    with open("models/svd_model.pkl", "rb") as f:
        model = pickle.load(f)

    # Get games the user has not rated
    rated_ids = reviews_df[reviews_df["user"] == user_name]["ID"].unique()
    unseen_ids = [gid for gid in games_df["id"].unique() if gid not in rated_ids]

    # Predict ratings
    predictions = [(gid, model.predict(user_name, gid).est) for gid in unseen_ids]
    top_preds = sorted(predictions, key=lambda x: x[1], reverse=True)[:top_k]

    # Retrieve game info
    top_game_ids = [gid for gid, _ in top_preds]
    recommended = games_df[games_df["id"].isin(top_game_ids)][["name", "rating", "id"]].copy()
    recommended["predicted_rating"] = [round(r, 2) for _, r in top_preds]

    return recommended.sort_values("predicted_rating", ascending=False)

In [ ]:
# ------------------------------
# Dash App
# ------------------------------
app = Dash(__name__, suppress_callback_exceptions=True)

# --- Global Styles ---
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>Board Game Recommender</title>
        {%favicon%}
        {%css%}
        <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600&display=swap" rel="stylesheet">
        <style>
            body {
                font-family: 'Inter', sans-serif;
                background-color: #f8f9fa;
                margin: 0;
                padding: 0;
            }
            .landing-container {
                display: flex;
                flex-direction: column;
                align-items: center;
                justify-content: center;
                height: 100vh;
                text-align: center;
            }
            .landing-container h1 {
                font-size: 3rem;
                margin-bottom: 0.5em;
            }
            .landing-container p {
                font-size: 1.2rem;
                color: #555;
                margin-bottom: 2em;
            }
            .button-row {
                display: flex;
                gap: 1.5em;
                flex-wrap: wrap;
                justify-content: center;
            }
            .start-button {
                font-size: 1rem;
                padding: 0.75em 1.5em;
                border: none;
                border-radius: 8px;
                background-color: #007BFF;
                color: white;
                cursor: pointer;
                transition: background-color 0.3s ease;
                min-width: 180px;
            }
            .start-button:hover {
                background-color: #0056b3;
            }
            .back-button-container {
                display: flex;
                justify-content: center;
                margin-top: 3em;
            }
            .back-button {
                font-size: 1rem;
                padding: 0.75em 1.5em;
                border: none;
                border-radius: 8px;
                background-color: #6c757d;
                color: white;
                cursor: pointer;
                transition: background-color 0.3s ease;
                min-width: 180px;
            }
            .back-button:hover {
                background-color: #5a6268;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

# --- Landing Layout ---
landing_layout = html.Div(className="landing-container", children=[
    html.H1("🎲 Welcome to the Board Game Recommender 🎲"),
    html.P("Discover games tailored to your preferences."),
    html.Div(className="button-row", children=[
        html.A(html.Button("Explore Dataset", className="start-button"), href="/exploration"),
        html.A(html.Button("Start Recommending", className="start-button"), href="/recommender")
    ])
])

# --- Recommender Layout ---
recommender_layout = html.Div(style={
    'display': 'flex',
    'flexDirection': 'column',
    'alignItems': 'center',
    'justifyContent': 'center',
    'padding': '5vh 2rem',
    'textAlign': 'center'
}, children=[
    html.Div("🕵️‍♂️", style={'fontSize': '5rem', 'marginBottom': '0.5rem'}),
    html.H2("Who are you?"),
    html.P("Enter your username to continue"),
    dcc.Input(
        id='user-name-input',
        type='text',
        placeholder='e.g. boardgamefan99',
        debounce=True,
        style={
            'padding': '0.6rem 1rem',
            'fontSize': '1rem',
            'borderRadius': '8px',
            'border': '1px solid #ccc',
            'width': '300px',
            'marginTop': '1rem'
        }
    ),

    html.Div(id='game-rating-inputs', style={'width': '100%', 'maxWidth': '600px'}),
    html.Div(style={'display': 'flex', 'gap': '1rem', 'marginTop': '1.5rem'}, children=[
        html.A(html.Button("← Go Back", className="back-button"), href="/"),
        html.Button("Continue", id='user-continue-button', n_clicks=0, style={
            'padding': '0.6rem 1.5rem',
            'fontSize': '1rem',
            'border': 'none',
            'borderRadius': '8px',
            'backgroundColor': '#007BFF',
            'color': 'white',
            'cursor': 'pointer'
        })
    ]),
    html.Div(id='user-error', style={'color': 'red', 'marginTop': '1rem'})
])

# --- Data Exploration Layout ---
exploration_layout = html.Div(style={
    'padding': '4rem 6vw',  # top/bottom 4rem, left/right 6% of viewport width
    'maxWidth': '1200px'
}, children=
    [
    html.H2("📊 Dataset Overview"),

    html.H4("ℹ️ TMI but..."),
    html.P([
        "Our dataset is self-collected and based on data from ",
        html.A("BoardGameGeek", href="https://boardgamegeek.com/", target="_blank"),
        ", the world’s largest online board game database and community. It contains",
        f" {len(games):,} board games including ratings, player requirements, age guidelines, and more."
    ]),
    html.P("BoardGameGeek hosts ratings, reviews, descriptions, and play guides for thousands of games, powered by contributions from over 2 million registered users. We could say it’s a go-to hub for board game enthusiasts around the world."),

    html.Br(),
    html.H4("🔍 TF-IDF Summary"),
    html.P("Ever wondered which words make board games truly stand out? The chart and cloud below show the most distinctive terms across all game descriptions, calculated using a method called TF-IDF. The higher the score, the more unique and important a word is to describing those games. Try adjusting the slider to explore the top keywords!"),

    html.Div(style={
        'backgroundColor': '#e7f1fb',
        'border': '1px solid #c3d9ec',
        'borderRadius': '12px',
        'padding': '1.5rem',
        'marginTop': '1rem',
        'marginBottom': '2rem',
        'boxShadow': '0 2px 5px rgba(0,0,0,0.05)'
    }, children=[
        html.Label("Select number of top terms:"),
        dcc.Slider(
            id='top-k-slider',
            min=5,
            max=50,
            step=1,
            value=25,
            marks={i: str(i) for i in range(5, 51, 5)},
            tooltip={"placement": "bottom", "always_visible": True}
        ),
        html.Br(),

        html.Div(style={'display': 'flex', 'justifyContent': 'space-between', 'gap': '2em'}, children=[

            # Left: bar chart
            html.Div(style={'flex': '1'}, children=[
                dcc.Graph(id='tfidf-bar-chart', style={'height': '280px'})
            ]),

            # Right: word cloud
            html.Div(style={'flex': '1', 'textAlign': 'center'}, children=[
                html.Img(id='tfidf-wordcloud',
                        src='data:image/png;base64,{}'.format(encoded_image),
                        style={'width': '100%', 'maxWidth': '100%'})
            ])
        ])
    ]),

    html.Br(),
    html.H4("🧠 Topic Distribution (LDA)"),
    html.P("Explore how topics are distributed across the board game descriptions. This interactive map lets you hover over topics, see top keywords, and better understand how the model organizes the data."),
    html.Div(style={
        'backgroundColor': '#e7f1fb',
        'border': '1px solid #c3d9ec',
        'borderRadius': '12px',
        'padding': '1.5rem',
        'marginTop': '1rem',
        'marginBottom': '2rem',
        'boxShadow': '0 2px 5px rgba(0,0,0,0.05)'
    }, children=[
        html.Iframe(
            src="/assets/lda_vis.html",
            style={
                "width": "100%",
                "height": "600px",
                "border": "none",
                "borderRadius": "12px"
            }
        )
    ]),

    html.Div(className="back-button-container", children=[
        html.A(html.Button("← Go Back", className="back-button"), href="/")
    ])
])

# --- Register User Layout ---
register_user_layout = html.Div(style={
    'display': 'flex',
    'flexDirection': 'column',
    'alignItems': 'center',
    'padding': '5vh 2rem',
    'textAlign': 'center'
}, children=[
    html.H2("Oops! Looks like you are new here...please register!"),

    # Username row
    html.Div(style={'display': 'flex', 'alignItems': 'center', 'gap': '1rem', 'marginBottom': '1.5rem'}, children=[
        html.Label("Username:", style={'fontWeight': 'bold', 'whiteSpace': 'nowrap'}),
        dcc.Input(
            id='new-username-input',
            type='text',
            placeholder='e.g. boardgamefan123',
            style={
                'padding': '0.6rem 1rem',
                'fontSize': '1rem',
                'borderRadius': '8px',
                'border': '1px solid #ccc',
                'width': '300px'
            }
        )
    ]),

    # Game selection row
    html.Div(style={'display': 'flex', 'alignItems': 'center', 'gap': '1rem', 'marginBottom': '1.5rem'}, children=[
        html.Label("Search for 5 games you like:", style={'fontWeight': 'bold', 'whiteSpace': 'nowrap'}),
        dcc.Dropdown(
            id='game-selection-dropdown',
            options=[
                {'label': row['name'], 'value': row['id']}
                for _, row in games.sort_values('name').iterrows()
            ],
            multi=True,
            placeholder="Start typing to find games…",
            style={'width': '500px'}
        )
    ]),

    # Buttons
    html.Div(style={'display': 'flex', 'gap': '1rem'}, children=[
        html.A(html.Button("← Go Back", className="back-button"), href="/new-user-choice"),
        html.Button("Register", id="register-button", n_clicks=0, style={
            'padding': '0.6rem 1.5rem',
            'fontSize': '1rem',
            'border': 'none',
            'borderRadius': '8px',
            'backgroundColor': '#007BFF',
            'color': 'white',
            'cursor': 'pointer'
        })
    ]),

    html.Div(id='register-error', style={'color': 'red', 'marginTop': '1rem'})
])

# --- Recommendation Options Layout ---
recommendation_options_layout = html.Div(style={
    'display': 'flex',
    'flexDirection': 'column',
    'alignItems': 'center',
    'padding': '5vh 2rem',
    'textAlign': 'center'
}, children=[
    html.H2("🧠 Choose a Recommendation Strategy"),
    html.P("Select an algorithm below to get your personalized board game recommendations."),

    html.Div(style={'display': 'flex', 'gap': '1.5rem', 'marginTop': '2rem'}, children=[
        html.Button("🔍 Content-Based Filtering (MMR)", id='algo-content', className='start-button'),
        html.Button("🤝 Collaborative Filtering", id='algo-collab', className='start-button'),
        html.Button("🧪 Hybrid Approach", id='algo-hybrid', className='start-button')
    ]),

    html.Div(className="back-button-container", children=[
        html.A(html.Button("← Go Back", className="back-button"), href="/recommender")
    ])
])

# --- New User Choice Layout ---
new_user_choice_layout = html.Div(style={
    'display': 'flex',
    'flexDirection': 'column',
    'alignItems': 'center',
    'justifyContent': 'center',
    'padding': '8vh 2rem',
    'textAlign': 'center'
}, children=[
    html.Div("✨", style={'fontSize': '4rem', 'marginBottom': '0.5rem'}),
    html.H2("Looks like it's your first time here!"),
    html.P("Would you like to create a profile for personalized recommendations or just see what's popular?"),

    html.Div(style={'display': 'flex', 'gap': '2rem', 'marginTop': '2rem'}, children=[
        html.Button("🎯 Create a Profile", id="go-to-register", n_clicks=0, style={
            'padding': '1rem 2rem',
            'fontSize': '1.1rem',
            'border': 'none',
            'borderRadius': '10px',
            'backgroundColor': '#007BFF',
            'color': 'white',
            'cursor': 'pointer',
            'minWidth': '220px'
        }),
        html.Button("🔥 Show Popular Games", id="go-to-popular", n_clicks=0, style={
            'padding': '1rem 2rem',
            'fontSize': '1.1rem',
            'border': 'none',
            'borderRadius': '10px',
            'backgroundColor': '#28a745',
            'color': 'white',
            'cursor': 'pointer',
            'minWidth': '220px'
        })
    ]),

    html.Div(className="back-button-container", children=[
        html.A(html.Button("← Go Back", className="back-button"), href="/recommender")
    ])
])

# --- Popular Games Layout ---
popular_games_layout = html.Div(style={
    'display': 'flex',
    'flexDirection': 'column',
    'alignItems': 'center',
    'padding': '5vh 2rem',
    'textAlign': 'center'
}, children=[
    html.H2("🔥 Popular Board Games"),
    html.Div(style={'width': '100%', 'maxWidth': '600px', 'marginBottom': '2rem'}, children=[
        html.Label("Select how many games to show:", style={'fontWeight': 'bold'}),
        dcc.Slider(
            id="top-n-slider",
            min=5,
            max=15,
            step=1,
            value=10,
            marks={i: str(i) for i in range(5, 16)},
            tooltip={"placement": "bottom", "always_visible": True},
            updatemode="drag"
        )
    ]),
    html.Div(id="popular-games-output", style={
        'width': '100%',
        'maxWidth': '800px'
    }),
    html.Div(className="back-button-container", children=[
        html.A(html.Button("← Go Back", className="back-button"), href="/new-user-choice")
    ])
])

# --- Content-Based MMR Layout ---
content_mmr_layout = html.Div(style={
    'display': 'flex',
    'flexDirection': 'column',
    'alignItems': 'center',
    'padding': '5vh 2rem',
    'textAlign': 'center'
}, children=[

    html.H2("🎯 Content-Based MMR Recommender"),
    html.P("Customize the parameters below to get personalized and diverse game recommendations."),

    html.Div([

        html.Div([
            html.Label("🎲 Max Playing Time (minutes):"),
            dcc.Slider(
                id="input-max-playingtime",
                min=10, max=300, step=10, value=60,
                marks={i: str(i) for i in range(30, 301, 30)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={"width": "100%", "maxWidth": "600px", "marginBottom": "1.5rem"}),

        html.Div([
            html.Label("👶 Minimum Age:"),
            dcc.Slider(
                id="input-min-age",
                min=3, max=18, step=1, value=10,
                marks={i: str(i) for i in range(3, 19, 3)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={"width": "100%", "maxWidth": "600px", "marginBottom": "1.5rem"}),

        html.Div([
            html.Label("👥 Minimum Players:"),
            dcc.Slider(
                id="input-min-players",
                min=1, max=10, step=1, value=2,
                marks={i: str(i) for i in range(1, 11)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={"width": "100%", "maxWidth": "600px", "marginBottom": "1.5rem"}),

        html.Div([
            html.Label("👥 Maximum Players:"),
            dcc.Slider(
                id="input-max-players",
                min=1, max=20, step=1, value=4,
                marks={i: str(i) for i in range(2, 21, 2)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={"width": "100%", "maxWidth": "600px", "marginBottom": "1.5rem"}),

        html.Div([
            html.Label("⭐ Minimum Rating:"),
            dcc.Slider(
                id="input-min-rating",
                min=1, max=10, step=1, value=6,
                marks={i: str(i) for i in range(1, 11)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={"width": "100%", "maxWidth": "600px", "marginBottom": "1.5rem"}),

        html.Div([
            html.Label("🔝 Number of Recommendations (Top-K):"),
            dcc.Slider(
                id="input-top-k",
                min=3, max=20, step=1, value=10,
                marks={i: str(i) for i in range(3, 21)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={"width": "100%", "maxWidth": "600px", "marginBottom": "1.5rem"}),

        html.Div([
            html.Label("⚖️ MMR Lambda (Diversity vs Relevance):"),
            dcc.Slider(
                id="input-lambda-param",
                min=0.0, max=1.0, step=0.05, value=0.7,
                marks={i / 10: str(i / 10) for i in range(0, 11)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={"width": "100%", "maxWidth": "600px", "marginBottom": "2rem"}),

        html.Button("🚀 Run Recommender", id="run-mmr-recommender", n_clicks=0, className="start-button")

    ], style={"width": "100%", "maxWidth": "650px"}),

    html.Hr(style={"margin": "3rem 0", "width": "100%"}),

    html.Div(id="cb-mmr-recommendation-output", style={"width": "100%", "maxWidth": "800px"}),
    html.Img(id="cb-mmr-plot-output", style={"marginTop": "2rem", "maxWidth": "100%"}),

    html.Div(className="back-button-container", children=[
        html.A(html.Button("← Go Back", className="back-button"), href="/recommendation-options")
    ])
])

# --- Hybrid Content-Based Layout ---
hybrid_layout = html.Div(style={
    'display': 'flex',
    'flexDirection': 'column',
    'alignItems': 'center',
    'padding': '5vh 2rem',
    'textAlign': 'center'
}, children=[

    html.H2("🧪 Hybrid Recommender"),
    html.P("This approach combines content-based filtering with rating prediction using similarity."),

    html.Div(style={"width": "100%", "maxWidth": "650px", "marginBottom": "2rem"}, children=[
        html.Label("🔍 Number of Similar Games to Consider (k):", style={'fontWeight': 'bold'}),
        dcc.Slider(
            id="input-k",
            min=1,
            max=20,
            step=1,
            value=10,
            marks={i: str(i) for i in range(1, 21)},
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ]),

    html.Button("🚀 Run Hybrid Recommender", id="run-hybrid-recommender", n_clicks=0, className="start-button"),

    html.Hr(style={"margin": "3rem 0", "width": "100%"}),

    html.Div(id="hybrid-recommendation-output", style={"width": "100%", "maxWidth": "800px"}),

    html.Div(className="back-button-container", children=[
        html.A(html.Button("← Go Back", className="back-button"), href="/recommendation-options")
    ])
])

# --- Collaborative Filtering Layout ---
collaborative_layout = html.Div(style={
    'display': 'flex',
    'flexDirection': 'column',
    'alignItems': 'center',
    'padding': '5vh 2rem',
    'textAlign': 'center'
}, children=[
    html.H2("🤝 Collaborative Filtering"),
    html.P("Select the method and number of recommendations."),

    html.Div([
        html.Label("📌 Choose a method:"),
        dcc.RadioItems(
            id="collab-method",
            options=[
                {"label": "KNN With Means", "value": "knn"},
                {"label": "SVD", "value": "svd"}
            ],
            value="knn",
            labelStyle={"display": "inline-block", "marginRight": "1rem"}
        )
    ], style={"marginBottom": "2rem"}),

    html.Div([
        html.Label("🔝 Number of Recommendations (Top-K):"),
        dcc.Slider(
            id="collab-top-k",
            min=3, max=20, step=1, value=10,
            marks={i: str(i) for i in range(3, 21)},
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ], style={"width": "100%", "maxWidth": "600px", "marginBottom": "2rem"}),

    html.Button("🚀 Run Collaborative Recommender", id="run-collab-recommender", n_clicks=0, className="start-button"),

    html.Hr(style={"margin": "3rem 0", "width": "100%"}),

    html.Div(id="collab-recommendation-output", style={"width": "100%", "maxWidth": "800px"}),

    html.Div(className="back-button-container", children=[
        html.A(html.Button("← Go Back", className="back-button"), href="/recommendation-options")
    ])
])

# --- App Layout ---
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    dcc.Store(id='stored-username', storage_type='session'),
    html.Div(id='page-content')
])

# --- Page Router ---
@app.callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == "/recommender":
        return recommender_layout
    elif pathname == "/exploration":
        return exploration_layout
    elif pathname == "/register-user":
        return register_user_layout
    elif pathname == "/new-user-choice":
        return new_user_choice_layout
    elif pathname == "/recommendation-options":
        return recommendation_options_layout
    elif pathname == "/popular":
        return popular_games_layout
    elif pathname == "/content-based-mmr":
        return content_mmr_layout
    elif pathname == "/hybrid":
        return hybrid_layout
    elif pathname == "/collaborative-filtering":
        return collaborative_layout
    else:
        return landing_layout

@app.callback(
    Output('tfidf-bar-chart', 'figure'),
    Input('top-k-slider', 'value')
)
def update_tfidf_bar_chart(top_k):
    fig, _ = plot_tfidf_summary(dictionary, X_tfidf, top_n=top_k)
    return fig

@app.callback(
    Output('url', 'pathname'),
    Output('user-error', 'children'),
    Output('stored-username', 'data'),
    Input('user-continue-button', 'n_clicks'),
    State('user-name-input', 'value'),
    prevent_initial_call=True
)
def route_existing_user(n_clicks, username):
    if n_clicks == 0:
        raise PreventUpdate

    if not username or username.strip() == "":
        return dash.no_update, "Please enter a valid username.", None

    username = username.strip()
    if username in reviews["user"].values:
        return "/recommendation-options", "", username
    else:
        return "/new-user-choice", "", username

@app.callback(
    Output('url', 'pathname', allow_duplicate=True),
    Output('register-error', 'children'),
    Input('register-button', 'n_clicks'),
    State('new-username-input', 'value'),
    State('game-selection-dropdown', 'value'),
    prevent_initial_call=True
)
def register_user(n_clicks, new_username, selected_games):
    global reviews

    if n_clicks == 0:
        raise PreventUpdate

    if not new_username or new_username.strip() == "":
        return dash.no_update, "Please enter a username."
    if not selected_games or len(selected_games) < 5:
        return dash.no_update, "Please select at least 5 games."

    default_rating = 9
    new_entries = pd.DataFrame({
        "user": [new_username.strip()] * len(selected_games),
        "ID": selected_games,
        "rating": [default_rating] * len(selected_games),
        "comment": [np.nan] * len(selected_games),
        "name": [games.loc[games["id"] == gid, "name"].values[0] for gid in selected_games]
    })

    reviews = pd.concat([reviews, new_entries], ignore_index=True)

    return "/recommendation-options", ""

@app.callback(
    Output('url', 'pathname', allow_duplicate=True),
    Input('go-to-register', 'n_clicks'),
    Input('go-to-popular', 'n_clicks'),
    prevent_initial_call=True
)
def route_from_choice(n_register, n_popular):
    ctx = callback_context.triggered_id
    if ctx == "go-to-register":
        return "/register-user"
    elif ctx == "go-to-popular":
        return "/popular"
    raise PreventUpdate

@app.callback(
    Output("popular-games-output", "children"),
    Input("url", "pathname"),
    Input("top-n-slider", "value"),
    prevent_initial_call=True
)
def display_popular_games(pathname, top_k):
    if pathname != "/popular":
        raise PreventUpdate

    df = get_diverse_popular_games(games, top_k=top_k)

    return [
        html.Div(style={
            'border': '1px solid #ccc',
            'borderRadius': '10px',
            'padding': '1rem',
            'marginBottom': '1rem',
            'backgroundColor': 'white',
            'boxShadow': '0 2px 6px rgba(0,0,0,0.05)',
            'textAlign': 'center'
        }, children=[
            html.H4(row["name"], style={'marginBottom': '0.5rem'}),
            html.Small(f"⭐ Average Rating: {row['avg_rating']:.2f} from {int(row['num_ratings'])} ratings", style={'color': '#777'})
        ])
        for _, row in df.iterrows()
    ]

@app.callback(
    Output("cb-mmr-recommendation-output", "children"),
    Output("cb-mmr-plot-output", "src"),
    Input("run-mmr-recommender", "n_clicks"),
    State("stored-username", "data"),
    State("input-max-playingtime", "value"),
    State("input-min-age", "value"),
    State("input-min-players", "value"),
    State("input-max-players", "value"),
    State("input-min-rating", "value"),
    State("input-top-k", "value"),
    State("input-lambda-param", "value"),
    prevent_initial_call=True
)
def generate_cb_mmr_output(n_clicks, username, max_playingtime, min_age, minplayers, maxplayers, min_rating, top_k, lambda_param):
    if not username:
        return html.Div("⚠️ Please enter a username.", style={"color": "red"}), None

    results, plot_base64 = recommend_content_based_mmr(
        user_name=username,
        games=games,
        reviews=reviews,
        top_k=top_k,
        max_playingtime=max_playingtime,
        min_age=min_age,
        minplayers=minplayers,
        maxplayers=maxplayers,
        min_rating=min_rating,
        plot=True
    )

    if results is None or results.empty:
        return html.Div("😕 No recommendations found. Try relaxing the filters.", style={"color": "orange"}), None

    cards = []
    for _, row in results.iterrows():
        cards.append(html.Div([
            html.H4(row["name"]),
            html.P(f"Similarity Score: {row['similarity']:.2f}"),
            html.P(row["explanation"])
        ], style={
            "border": "1px solid #ccc",
            "borderRadius": "10px",
            "padding": "1rem",
            "marginBottom": "1rem",
            "backgroundColor": "#fefefe",
            "boxShadow": "0 2px 5px rgba(0,0,0,0.05)"
        }))

    img_src = f"data:image/png;base64,{plot_base64}" if plot_base64 else None
    return cards, img_src

@app.callback(
    Output("hybrid-recommendation-output", "children"),
    Input("run-hybrid-recommender", "n_clicks"),
    State("stored-username", "data"),
    State("input-k", "value"),
    prevent_initial_call=True
)
def run_hybrid_recommender(n_clicks, username, k):
    if not username:
        return html.Div("⚠️ Username not found. Please go back and enter your name.", style={"color": "red"})

    pred_df = predict_all_ratings(user_name=username, games_df=games, reviews_df=reviews, k=k)

    if isinstance(pred_df, str):  # error message
        return html.Div(f"⚠️ {pred_df}", style={"color": "orange"})

    cards = []
    for _, row in pred_df.iterrows():
        cards.append(html.Div([
            html.H4(row["name"]),
            html.P(f"Predicted Rating: {row['predicted_rating']}"),
            html.P(row["explanation"])
        ], style={
            "border": "1px solid #ccc",
            "borderRadius": "10px",
            "padding": "1rem",
            "marginBottom": "1rem",
            "backgroundColor": "#fefefe",
            "boxShadow": "0 2px 5px rgba(0,0,0,0.05)"
        }))

    return cards

@app.callback(
    Output('url', 'pathname', allow_duplicate=True),
    Input('algo-content', 'n_clicks'),
    Input('algo-collab', 'n_clicks'),
    Input('algo-hybrid', 'n_clicks'),
    prevent_initial_call=True
)
def go_to_algo(n1, n2, n3):
    ctx = callback_context.triggered_id
    if ctx == "algo-content":
        return "/content-based-mmr"
    elif ctx == "algo-collab":
        return "/collaborative-filtering"
    elif ctx == "algo-hybrid":
        return "/hybrid"
    raise PreventUpdate

@app.callback(
    Output("collab-recommendation-output", "children"),
    Input("run-collab-recommender", "n_clicks"),
    State("collab-method", "value"),
    State("collab-top-k", "value"),
    State("stored-username", "data"),
    prevent_initial_call=True
)
def run_collaborative_recommender(n_clicks, method, top_k, username):
    if not username:
        return html.Div("⚠️ Username not found. Please go back and enter your name.", style={"color": "red"})

    if method == "knn":
        results = recommend_with_knn(username, games, reviews, top_k)
    elif method == "svd":
        results = recommend_with_svd(username, games, reviews, top_k)

    else:
        return html.Div("⚠️ Unknown method selected.", style={"color": "red"})

    if results is None or results.empty:
        return html.Div("😕 No recommendations found.", style={"color": "orange"})

    cards = []
    for _, row in results.iterrows():
        cards.append(html.Div([
            html.H4(row["name"]),
            html.P(f"Predicted Rating: {row['predicted_rating']}")
        ], style={
            "border": "1px solid #ccc",
            "borderRadius": "10px",
            "padding": "1rem",
            "marginBottom": "1rem",
            "backgroundColor": "#fefefe",
            "boxShadow": "0 2px 5px rgba(0,0,0,0.05)"
        }))

    return cards

if __name__ == '__main__':
    app.run(debug=True)